<a href="https://colab.research.google.com/github/mingd00/aivle-miniproj6/blob/main/2%EC%B0%A8.%20%EC%9D%91%EA%B8%89%EC%83%81%ED%99%A9%20%EC%9D%B8%EC%8B%9D%20%EB%B0%8F%20%EC%9D%91%EA%B8%89%EC%8B%A4%20%EC%97%B0%EA%B3%84/4-2.%20%ED%86%B5%ED%95%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **응급상황 자동 인식 및 응급실 연계 서비스**
### **통합 - 단위 테스트 및 파이프라인 코드를 작성**

## **1.환경설정**

#### 1) 구글 드라이브 연결

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
path = '/content/drive/MyDrive/project6_2/'

#### 2) 라이브러리

In [66]:
!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

In [67]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import openai
from openai import OpenAI
import json
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 모듈 import
import emergency

## **2. 단위 테스트**

### (1) open ai key 등록

In [69]:
openai.api_key = emergency.load_keys(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

### (2) audio to text

In [70]:
audio_path = path + 'total_audio/'
file_list = os.listdir(audio_path)

# 빈 데이터프레임 선언
df = pd.DataFrame(columns=['filename', 'text'])

# 반복문 수행하면서 오디오 변환
for filename in file_list:
    try:
        text = emergency.audio2text(audio_path, filename)
        df.loc[len(df)] = [filename, text]
    except Exception as e:
        print(f"오류 발생: {filename} - {e}")

df

,filename,text
0,audio16.m4a,119죠? 제가 운전을 하다가 쓰러진 사람을 봤어요. 의식이 아예 없는 것 같아요....
1,audio14.m4a,119죠? 제가 지금 열이 40도까지 올랐어요. 머리가 너무 뜨겁고 어지럽고 움직이...
2,audio18.m4a,지하철에 사람이 쓰러졌어요. 갑자기 심장을 움켜쥐더니 쓰러졌습니다. 얼른 와주세요.\n
3,audio17.m4a,생선 손질을 하다가 손가락이 베었습니다. 손가락이 잘려서 피가 너무 많이 났는데 잘...
4,audio15.m4a,119죠. 동생이 숨을 못 쉴 정도로 기침을 하고 있어요. 어떻게 하면 좋을까요?\n
5,audio11.m4a,119죠? 뜨거운 물을 끓이다가 손에 데였어요 화상을 입은 것 같은데 어떻게 해야 ...
6,audio13.m4a,몇 시간 전에 머리를 부딪혔는데 속이 토할 것 같이 너무 안좋고 어지러워요. 도와주...
7,audio12.m4a,일일 보조. 뜨거운 물을 끓이다가 손에 데웠어요. 화상을 입은 것 같은데 어떻게 해...
8,5등급(1).m4a,제 조카가 벌에 쏘였는데요 소인 부위가 조금 붓고 아프다고 합니다. 숨 쉬는데 문제...
9,2등급(2).m4a,제 남동생이 축구를 하다가 코를 다쳤어요. 코피가 계속 나고 지혈을 했는데도 멈추질...


### (3) text summary

In [71]:
role = '당신은 응급 상황을 20글자 내외로 요약해주는 어시스턴트 입니다.'
for text in df['text']:
  summary = emergency.text2summary(role, text)
  df.loc[df['text'] == text, 'summary'] = summary

df

,filename,text,summary
0,audio16.m4a,119죠? 제가 운전을 하다가 쓰러진 사람을 봤어요. 의식이 아예 없는 것 같아요....,119에 신고할 상황: 운전 중 쓰러진 의식 없는 사람 발견하여 긴급 구조 요청함.
1,audio14.m4a,119죠? 제가 지금 열이 40도까지 올랐어요. 머리가 너무 뜨겁고 어지럽고 움직이...,상당히 높은 열이 올라가서 머리가 뜨거우며 어지러워서 움직일 수 없는 상황이다.
2,audio18.m4a,지하철에 사람이 쓰러졌어요. 갑자기 심장을 움켜쥐더니 쓰러졌습니다. 얼른 와주세요.\n,지하철에서 갑자기 심장 마비로 인해 사람이 쓰러진 상황이 발생했으며 신속한 도움이 ...
3,audio17.m4a,생선 손질을 하다가 손가락이 베었습니다. 손가락이 잘려서 피가 너무 많이 났는데 잘...,생선 손질 중에 손가락 다친 상황. 잘린 손가락에 대해 도움이 필요함.
4,audio15.m4a,119죠. 동생이 숨을 못 쉴 정도로 기침을 하고 있어요. 어떻게 하면 좋을까요?\n,동생이 기침으로 숨을 쉬기 어렵다고 할 때 대처 방법에 대해 물었어요.
5,audio11.m4a,119죠? 뜨거운 물을 끓이다가 손에 데였어요 화상을 입은 것 같은데 어떻게 해야 ...,"""119에 전화해서 뜨거운 물로 화상을 입은 손 상처에 대해 도움 요청 중입니다."""
6,audio13.m4a,몇 시간 전에 머리를 부딪혔는데 속이 토할 것 같이 너무 안좋고 어지러워요. 도와주...,머리를 부딪혀서 안좋고 어지러운 상황에서 도움이 필요하다고 합니다.
7,audio12.m4a,일일 보조. 뜨거운 물을 끓이다가 손에 데웠어요. 화상을 입은 것 같은데 어떻게 해...,뜨거운 물을 끓이다가 손에 데웠고 화상을 입은 상황에서 도움을 요청하는 것 같아요.
8,5등급(1).m4a,제 조카가 벌에 쏘였는데요 소인 부위가 조금 붓고 아프다고 합니다. 숨 쉬는데 문제...,조카가 벌에 쏘여 소인 부위가 붓고 아프지만 숨 쉬는 데는 문제가 없는 상황.
9,2등급(2).m4a,제 남동생이 축구를 하다가 코를 다쳤어요. 코피가 계속 나고 지혈을 했는데도 멈추질...,"남동생이 축구 중에 코를 다쳤고, 출혈이 멈추지 않습니다."


### (4) 응급실 등급분류

In [72]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_directory = path + "fine_tuned_bert"

# 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
model.to(device)
# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [73]:
# text 예제 1개 test
text = "제 남동생이 축구를 하다가 코를 다쳤어요. 코피가 계속 나고 지혈을 했는데도 멈추질 않아요."
predicted_class, probabilities = emergency.predict(text, model, tokenizer, device)

print(f"예측된 클래스: {predicted_class}")
print(f"예측된 클래스 이름: {predicted_class+1}등급")
print(f"클래스별 확률: {probabilities}")

예측된 클래스: 0
예측된 클래스 이름: 1등급
클래스별 확률: tensor([[9.6851e-01, 9.0983e-04, 4.2948e-05, 4.5727e-04, 3.0083e-02]])


In [84]:
# 전체 df에 예측값 업데이트
for summary in df['summary']:
  predicted_class, probabilities = emergency.predict(summary, model, tokenizer, device)
  df.loc[df['summary'] == summary, 'predicted_class'] = f"{predicted_class+1}등급"

df

,filename,text,summary,predicted_class,lat,lng
0,audio16.m4a,119죠? 제가 운전을 하다가 쓰러진 사람을 봤어요. 의식이 아예 없는 것 같아요....,119에 신고할 상황: 운전 중 쓰러진 의식 없는 사람 발견하여 긴급 구조 요청함.,2등급,35.147037,128.567729
1,audio14.m4a,119죠? 제가 지금 열이 40도까지 올랐어요. 머리가 너무 뜨겁고 어지럽고 움직이...,상당히 높은 열이 올라가서 머리가 뜨거우며 어지러워서 움직일 수 없는 상황이다.,2등급,37.537358,126.844864
2,audio18.m4a,지하철에 사람이 쓰러졌어요. 갑자기 심장을 움켜쥐더니 쓰러졌습니다. 얼른 와주세요.\n,지하철에서 갑자기 심장 마비로 인해 사람이 쓰러진 상황이 발생했으며 신속한 도움이 ...,1등급,36.431041,127.407191
3,audio17.m4a,생선 손질을 하다가 손가락이 베었습니다. 손가락이 잘려서 피가 너무 많이 났는데 잘...,생선 손질 중에 손가락 다친 상황. 잘린 손가락에 대해 도움이 필요함.,4등급,37.837564,127.153316
4,audio15.m4a,119죠. 동생이 숨을 못 쉴 정도로 기침을 하고 있어요. 어떻게 하면 좋을까요?\n,동생이 기침으로 숨을 쉬기 어렵다고 할 때 대처 방법에 대해 물었어요.,4등급,37.479625,126.936583
5,audio11.m4a,119죠? 뜨거운 물을 끓이다가 손에 데였어요 화상을 입은 것 같은데 어떻게 해야 ...,"""119에 전화해서 뜨거운 물로 화상을 입은 손 상처에 대해 도움 요청 중입니다.""",4등급,37.565110,128.540561
6,audio13.m4a,몇 시간 전에 머리를 부딪혔는데 속이 토할 것 같이 너무 안좋고 어지러워요. 도와주...,머리를 부딪혀서 안좋고 어지러운 상황에서 도움이 필요하다고 합니다.,4등급,37.278166,127.382000
7,audio12.m4a,일일 보조. 뜨거운 물을 끓이다가 손에 데웠어요. 화상을 입은 것 같은데 어떻게 해...,뜨거운 물을 끓이다가 손에 데웠고 화상을 입은 상황에서 도움을 요청하는 것 같아요.,4등급,37.378598,128.206139
8,5등급(1).m4a,제 조카가 벌에 쏘였는데요 소인 부위가 조금 붓고 아프다고 합니다. 숨 쉬는데 문제...,조카가 벌에 쏘여 소인 부위가 붓고 아프지만 숨 쉬는 데는 문제가 없는 상황.,2등급,35.582854,126.838104
9,2등급(2).m4a,제 남동생이 축구를 하다가 코를 다쳤어요. 코피가 계속 나고 지혈을 했는데도 멈추질...,"남동생이 축구 중에 코를 다쳤고, 출혈이 멈추지 않습니다.",3등급,35.866235,127.027449


### (5) 응급실추천

In [85]:
data = pd.read_csv(path + '응급실 정보.csv')

c_id = 'eqcpprijve'
c_key = 'ArjbnLuOxq1VUykF4LPl0owjjv25lE61cSmytMVz'

In [86]:
# 예제 1개 test
emergency.recommend_hospital(data, 34.8092145829474, 126.4163391650001, 0.1, c_id, c_key)

[['목포한국병원', 0], ['목포기독병원', 1360], ['목포시의료원', 1455]]

In [87]:
# 위도, 경도 임의로 추가
df['lat'] = np.random.choice(data['위도'], len(df))
df['lng'] = np.random.choice(data['경도'], len(df))
df.head()

,filename,text,summary,predicted_class,lat,lng
0,audio16.m4a,119죠? 제가 운전을 하다가 쓰러진 사람을 봤어요. 의식이 아예 없는 것 같아요....,119에 신고할 상황: 운전 중 쓰러진 의식 없는 사람 발견하여 긴급 구조 요청함.,2등급,36.858487,126.796365
1,audio14.m4a,119죠? 제가 지금 열이 40도까지 올랐어요. 머리가 너무 뜨겁고 어지럽고 움직이...,상당히 높은 열이 올라가서 머리가 뜨거우며 어지러워서 움직일 수 없는 상황이다.,2등급,36.323025,127.450095
2,audio18.m4a,지하철에 사람이 쓰러졌어요. 갑자기 심장을 움켜쥐더니 쓰러졌습니다. 얼른 와주세요.\n,지하철에서 갑자기 심장 마비로 인해 사람이 쓰러진 상황이 발생했으며 신속한 도움이 ...,1등급,37.199230,127.894252
3,audio17.m4a,생선 손질을 하다가 손가락이 베었습니다. 손가락이 잘려서 피가 너무 많이 났는데 잘...,생선 손질 중에 손가락 다친 상황. 잘린 손가락에 대해 도움이 필요함.,4등급,37.141228,126.885910
4,audio15.m4a,119죠. 동생이 숨을 못 쉴 정도로 기침을 하고 있어요. 어떻게 하면 좋을까요?\n,동생이 기침으로 숨을 쉬기 어렵다고 할 때 대처 방법에 대해 물었어요.,4등급,37.341829,126.885289


In [88]:
# 전체 df에서 값 호출한 후 추천
for idx, row in df[:3].iterrows():
  print(emergency.recommend_hospital(data, row['lat'], row['lng'], 0.1, c_id, c_key))

[]
[['대전화병원', 1673], ['학교법인가톨릭학원가톨릭대학교대전성모병원', 3441], ['충남대학교병원', 3568]]
[]


## **3. 파이프라인**

In [91]:
def pipeline(path, c_id, c_key, data, range, device):
  ## 1. open ai key 등록
  openai.api_key = emergency.load_keys(path + 'api_key.txt')
  os.environ['OPENAI_API_KEY'] = openai.api_key

  ## 2. audio to text
  audio_path = path + 'total_audio/'
  file_list = os.listdir(audio_path)

  # 빈 데이터프레임 선언
  df = pd.DataFrame(columns=['filename', 'text'])

  # 반복문 수행하면서 오디오 변환
  for filename in file_list:
      try:
          text = emergency.audio2text(audio_path, filename)
          df.loc[len(df)] = [filename, text]
      except Exception as e:
          print(f"오류 발생: {filename} - {e}")

  ## 3. text summary
  for text in df['text']:
    role = '당신은 입력된 텍스트를 가장 간결하고 핵심적인 표현으로 요약하는 어시스턴트입니다. 불필요한 설명을 생략하고 핵심적인 정보를 짧은 문장으로 요약하세요.'
    summary = emergency.text2summary(role, text)
    df.loc[df['text'] == text, 'summary'] = summary

  ## 4. 응급실 등급 분류
  import torch
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  save_directory = path + "fine_tuned_bert"
  # 모델, 토크나이저 호출
  model = AutoModelForSequenceClassification.from_pretrained(save_directory)
  model.to(device)
  tokenizer = AutoTokenizer.from_pretrained(save_directory)
  # 등급 분류
  for summary in df['summary']:
    predicted_class, probabilities = emergency.predict(summary, model, tokenizer, device)
    df.loc[df['summary'] == summary, 'predicted_class'] = f"{predicted_class+1}등급"

  ## 5. 응급실 추천
  data = pd.read_csv(path + '응급실 정보.csv')
  # 위도, 경도 임의로 추가
  df['lat'] = np.random.choice(data['위도'], len(df))
  df['lng'] = np.random.choice(data['경도'], len(df))
  # 가장 가까운 3개의 병원 출력
  for idx, row in df.iterrows():
    # 만약 4, 5 등급이라면 따로 메세지 출력
    if row['predicted_class'] == '4등급' or row['predicted_class'] == '5등급':
      print('가정에서 푹 쉬시면서 몸조리 잘 하세요')
    else:
      print(emergency.recommend_hospital(data, row['lat'], row['lng'], range, c_id, c_key))

In [92]:
path = '/content/drive/MyDrive/project6_2/'
c_id = 'eqcpprijve'
c_key = 'ArjbnLuOxq1VUykF4LPl0owjjv25lE61cSmytMVz'
data = pd.read_csv(path + '응급실 정보.csv')
range = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline(path, c_id, c_key, data, range, device)

[['하늘정형외과의원', 7921]]
[['충주미래병원', 3090], ['건국대학교충주병원', 3514], ['충청북도충주의료원', 6460]]
가정에서 푹 쉬시면서 몸조리 잘 하세요
[['의료법인대산의료재단익산병원', 5767], ['원광대학교병원', 6399]]
[]
[['홍천아산병원', 9926]]
가정에서 푹 쉬시면서 몸조리 잘 하세요
가정에서 푹 쉬시면서 몸조리 잘 하세요
가정에서 푹 쉬시면서 몸조리 잘 하세요
[['학교법인)동의병원', 954], ['학교법인 춘해병원', 3726], ['의료법인 온그룹의료재단 온종합병원', 4191]]
[]
가정에서 푹 쉬시면서 몸조리 잘 하세요
[['참조은병원', 9749]]
가정에서 푹 쉬시면서 몸조리 잘 하세요
가정에서 푹 쉬시면서 몸조리 잘 하세요
가정에서 푹 쉬시면서 몸조리 잘 하세요
가정에서 푹 쉬시면서 몸조리 잘 하세요
가정에서 푹 쉬시면서 몸조리 잘 하세요
가정에서 푹 쉬시면서 몸조리 잘 하세요
가정에서 푹 쉬시면서 몸조리 잘 하세요
[['원광대학교병원', 4602], ['의료법인대산의료재단익산병원', 5293]]
[['재단법인아산사회복지재단정읍아산병원', 21860]]
[]
가정에서 푹 쉬시면서 몸조리 잘 하세요
[['새금산병원', 11424]]
[['보성아산병원', 6998]]
[['첨단종합병원', 10416], ['KS병원', 11275], ['광주보훈병원', 11292]]
가정에서 푹 쉬시면서 몸조리 잘 하세요
